In [8]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import load_img, img_to_array

import numpy as np


# Model definition
def create_model():
    input_hair = Input(shape=(64, 64, 3))
    input_skin = Input(shape=(64, 64, 3))

    x = Conv2D(32, (3, 3), activation='relu')(input_hair)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(64, (3, 3), activation='relu')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Flatten()(x)
    out_hair = Dense(4, activation='softmax')(x)

    y = Conv2D(32, (3, 3), activation='relu')(input_skin)
    y = MaxPooling2D((2, 2))(y)
    y = Conv2D(64, (3, 3), activation='relu')(y)
    y = MaxPooling2D((2, 2))(y)
    y = Flatten()(y)
    out_skin = Dense(3, activation='softmax')(y)

    model = Model(inputs=[input_hair, input_skin], outputs=[out_hair, out_skin])

    model.compile(optimizer='adam', 
                  loss=['sparse_categorical_crossentropy', 'sparse_categorical_crossentropy'], 
                  metrics=['accuracy'])

    return model

# Data loading
def load_data(hair_dir, skin_dir):
    datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

    train_hair_generator = datagen.flow_from_directory(
        hair_dir,
        target_size=(64, 64),
        batch_size=32,
        class_mode='binary',
        subset='training')

    validation_hair_generator = datagen.flow_from_directory(
        hair_dir,
        target_size=(64, 64),
        batch_size=32,
        class_mode='binary',
        subset='validation')

    train_skin_generator = datagen.flow_from_directory(
        skin_dir,
        target_size=(64, 64),
        batch_size=32,
        class_mode='binary',
        subset='training')

    validation_skin_generator = datagen.flow_from_directory(
        skin_dir,
        target_size=(64, 64),
        batch_size=32,
        class_mode='binary',
        subset='validation')

    return train_hair_generator, validation_hair_generator, train_skin_generator, validation_skin_generator

# Model training
# Model training
def train_model(model, train_hair_generator, train_skin_generator, validation_hair_generator, validation_skin_generator):
    def multi_generator(generator1, generator2):
        while True:
            x1, y1 = next(generator1)
            x2, y2 = next(generator2)
            yield [x1, x2], [y1, y2]

    train_multi_generator = multi_generator(train_hair_generator, train_skin_generator)
    validation_multi_generator = multi_generator(validation_hair_generator, validation_skin_generator)

    model.fit(train_multi_generator,
              steps_per_epoch=len(train_hair_generator),
              validation_data=validation_multi_generator,
              validation_steps=len(validation_hair_generator),
              epochs=20,verbose=2)

# Paths to image directories
hair_dir = '/content/drive/MyDrive/Fiverr work/color_detection/colordataset/hairs'
skin_dir = '/content/drive/MyDrive/Fiverr work/color_detection/colordataset/train'

# Load data
train_hair_generator, validation_hair_generator, train_skin_generator, validation_skin_generator = load_data(hair_dir, skin_dir)

# Create and train model
model = create_model()
train_model(model, train_hair_generator, train_skin_generator, validation_hair_generator, validation_skin_generator)


Found 195 images belonging to 4 classes.
Found 46 images belonging to 4 classes.
Found 201 images belonging to 3 classes.
Found 49 images belonging to 3 classes.
Epoch 1/20
7/7 - 115s - loss: 2.3822 - dense_loss: 1.3823 - dense_1_loss: 0.9999 - dense_accuracy: 0.3692 - dense_1_accuracy: 0.4577 - val_loss: 2.2058 - val_dense_loss: 1.2633 - val_dense_1_loss: 0.9425 - val_dense_accuracy: 0.4130 - val_dense_1_accuracy: 0.5102 - 115s/epoch - 16s/step
Epoch 2/20
7/7 - 3s - loss: 2.1558 - dense_loss: 1.2550 - dense_1_loss: 0.9009 - dense_accuracy: 0.3641 - dense_1_accuracy: 0.5174 - val_loss: 2.0419 - val_dense_loss: 1.1021 - val_dense_1_loss: 0.9398 - val_dense_accuracy: 0.5435 - val_dense_1_accuracy: 0.5714 - 3s/epoch - 500ms/step
Epoch 3/20
7/7 - 4s - loss: 1.8815 - dense_loss: 1.0265 - dense_1_loss: 0.8550 - dense_accuracy: 0.5590 - dense_1_accuracy: 0.6020 - val_loss: 1.7096 - val_dense_loss: 0.8861 - val_dense_1_loss: 0.8236 - val_dense_accuracy: 0.6522 - val_dense_1_accuracy: 0.5714 - 

In [11]:

# Load and preprocess the images
def preprocess_image(image_path):
    image = load_img(image_path, target_size=(64, 64))  # load and resize the image
    image = img_to_array(image)  # convert the image to a numpy array
    image = image / 255.0  # rescale the image
    image = np.expand_dims(image, axis=0)  # add an extra dimension for the batch size
    return image

hair_image_path = '/content/drive/MyDrive/Fiverr work/color_detection/colordataset/hairs/black hairs/data111.jpg'
skin_image_path = '/content/drive/MyDrive/Fiverr work/color_detection/colordataset/train/white/data127.jpg'

hair_image = preprocess_image(hair_image_path)
skin_image = preprocess_image(skin_image_path)

# Predict with the model
hair_prediction, skin_prediction = model.predict([hair_image, skin_image])

# Get the class names
hair_class_names = ['bald', 'black hairs',"red hair","white hairs"]
skin_class_names = ["black skin","light brown", "white skin"]

# Get the predicted classes
hair_predicted_class = hair_class_names[np.argmax(hair_prediction)]
skin_predicted_class = skin_class_names[np.argmax(skin_prediction)]

print("Hair prediction: ", hair_predicted_class)
print("Skin prediction: ", skin_predicted_class)


1/1 [==============================] - 0s 24ms/step
Hair prediction:  black hairs
Skin prediction:  white


In [15]:
import tensorflowjs as tfjs

def save_model_tfjs(model, path):
    tfjs.converters.save_keras_model(model, path)

# Save the model
save_model_tfjs(model, '/content/drive/MyDrive/Fiverr work/color_detection/js_based_model')


In [13]:
!pip install tensorflowjs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.1/85.1 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.9/189.9 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 83.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 5.8 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 23.1
    Uninstalling packaging-23.1:
      Successfully uninstalled packaging-23.1
  Attempting uninstall: flax
    Found existing installation: flax 0.6.9
    Uninstalling flax-0.6.9:
      Successfully uninstalled flax-0.6.9
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
statsmodels 0.13.5 requires packaging>=21.3, but you have packaging 20.9 which is incompatible.
xar

In [ ]:
import tensorflow as tf
from tensorflow import keras
import tensorflowjs as tfjs

tfjs.converters.save_keras_model(model, 'js_based_model')
